# 이미지 분석 데이터 검색

이 노트북에서는 이미지 분석이 포함된 데이터를 키워드, 벡터, 하이브리드 검색으로 활용합니다.

## 0. 필요한 라이브러리 설치

In [6]:
##############################################
# 0. 라이브러리 설치 (최초 1회 실행)
##############################################
%pip install --quiet python-dotenv azure-identity azure-search-documents openai

Note: you may need to restart the kernel to use updated packages.


## 1. 환경 변수 설정

In [7]:
import os
from dotenv import load_dotenv

##############################################
# 1. 환경 변수 로드 및 확인
##############################################
load_dotenv()

# Azure AI Search
AZURE_SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
AZURE_SEARCH_INDEX_NAME = os.getenv("AZURE_SEARCH_INDEX_NAME")

# Azure OpenAI
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_EMBEDDING_DEPLOYMENT = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-01")

print(f"Search Endpoint: {AZURE_SEARCH_SERVICE_ENDPOINT}")
print(f"Search Index: {AZURE_SEARCH_INDEX_NAME}")
print(f"OpenAI Endpoint: {AZURE_OPENAI_ENDPOINT}")
print(f"Embedding Model: {AZURE_OPENAI_EMBEDDING_DEPLOYMENT}")

Search Endpoint: https://ai-search-commercial-1023.search.windows.net
Search Index: products-index
OpenAI Endpoint: https://open-ai-seoulcentral-1023.openai.azure.com/
Embedding Model: text-embedding-3-large


## 2. 클라이언트 초기화

In [ ]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.search.documents import SearchClient
from openai import AzureOpenAI
from IPython.display import display, HTML
import os, shutil

##############################################
# 2-0. Azure 로그인을 위한 환경변수 등록 (az cli 경로 못찾는 경우)
##############################################
# AZ_DIR = "/opt/homebrew/bin" # 변경 필요
# os.environ["PATH"] = f"{AZ_DIR}:" + os.environ["PATH"]
# print("az which:", shutil.which("az"))

##############################################
# 2-1. Azure 인증 및 클라이언트 생성
##############################################
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(
    credential,
    "https://cognitiveservices.azure.com/.default"
)

search_client = SearchClient(
    endpoint=AZURE_SEARCH_SERVICE_ENDPOINT,
    index_name=AZURE_SEARCH_INDEX_NAME,
    credential=credential
)

openai_client = AzureOpenAI(
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_ad_token_provider=token_provider
)

##############################################
# 2-2. 임베딩 생성 함수
##############################################
def get_embedding(text):
    response = openai_client.embeddings.create(
        input=text,
        model=AZURE_OPENAI_EMBEDDING_DEPLOYMENT
    )
    return response.data[0].embedding

print("클라이언트 초기화 완료")

az which: /opt/homebrew/bin/az
클라이언트 초기화 완료


## 3. 결과 표시 헬퍼 함수

In [9]:
##############################################
# 3. 결과 표시 헬퍼 함수
##############################################
def display_search_results(results, title="검색 결과"):
    """검색 결과를 보기 좋게 표시합니다."""
    print(f"\n{'='*80}")
    print(f"[{title}]")
    print(f"{'='*80}\n")
    
    for idx, result in enumerate(results, 1):
        score = result.get('@search.score', 'N/A')
        reranker_score = result.get('@search.reranker_score')
        
        print(f"{idx}. {result['name']} ({result['brand']})")
        print(f"   가격: {result['price']:,.0f}원")
        print(f"   검색 점수: {score}")
        
        if reranker_score:
            print(f"   Semantic Re-ranker Score: {reranker_score}")
        
        # 이미지 캡션 표시
        if result.get('imageCaption'):
            print(f"   💬 이미지 캡션: {result['imageCaption']}")
        
        # 이미지 태그 표시
        if result.get('imageTags'):
            tags = ', '.join(result['imageTags'][:5])
            print(f"   🏷️  태그: {tags}")
        
        # 제품 설명
        print(f"   📝 설명: {result['description'][:100]}...")
        
        # 이미지 표시
        if result.get('imageUrl'):
            display(HTML(f'<img src="{result["imageUrl"]}" width="300" style="margin: 10px 0;" />'))
        
        print()

print("헬퍼 함수 정의 완료")

헬퍼 함수 정의 완료


## 4. 이미지 정보 기반 검색

검색 범위: imageCaption, imageDescription, imageTags

In [10]:
##############################################
# 4. 이미지 캡션 기반 검색
##############################################
search_query = "검정색 가방"

results = search_client.search(
    search_text=search_query,
    top=3,
    search_fields=["imageCaption", "imageDescription", "imageTags"],  # 이미지 캡션과 설명만 검색
    select=["id", "name", "brand", "price", "description", "imageUrl", "imageCaption", "imageDescription", "imageTags"]
)

print(f"🔍 검색어: '{search_query}'")
print(f"💬 검색 필드: imageCaption, imageDescription (이미지 분석 결과만)")

display_search_results(results, "이미지 캡션/설명 검색 결과")

🔍 검색어: '검정색 가방'
💬 검색 필드: imageCaption, imageDescription (이미지 분석 결과만)

[이미지 캡션/설명 검색 결과]

1. 슬림핏 청바지 (데님스토리)
   가격: 89,000원
   검색 점수: 2.9245477
   💬 이미지 캡션: 3개의 청바지가 검정 배경 위에 나란히 놓여져 있습니다.
   🏷️  태그: 청바지, 패션, 옷, 데님, 카주얼
   📝 설명: 스트레치 소재가 함유된 슬림핏 청바지로 활동성이 뛰어나며 세련된 실루엣을 연출합니다....



2. 가죽 크로스백 (럭셔리레더)
   가격: 189,000원
   검색 점수: 2.036945
   💬 이미지 캡션: 오렌지 컬러의 가죽과 라탄 소재로 만든 가방.
   🏷️  태그: 가방, 디자인, 라탄, 가죽, 오렌지
   📝 설명: 고급 천연 가죽으로 제작된 크로스백입니다. 심플한 디자인으로 어떤 스타일과도 잘 어울립니다....



3. 숄더백 (어바에센셜)
   가격: 159,000원
   검색 점수: 1.647151
   💬 이미지 캡션: A black leather quilted handbag with gold accents.
   🏷️  태그: bag, leather, black, quilted, gold
   📝 설명: 심플한 디자인의 가죽 숄더백입니다. 수납공간이 넉넉하고 일상 사용에 편리합니다....


## 5. 이미지 정보를 포함한 하이브리드 검색

검색 범위: name, description, imageCaption, imageDescription, imageTags + 벡터 임베딩

In [11]:
from azure.search.documents.models import VectorizedQuery

##############################################
# 5. 하이브리드 검색
##############################################
search_query = "검정색 가죽 가방 찾아줘"

# 검색어를 벡터로 변환
query_vector = get_embedding(search_query)

vector_query = VectorizedQuery(
    vector=query_vector,
    k_nearest_neighbors=5,
    fields="descriptionVector"
)

results = search_client.search(
    search_text=search_query,
    vector_queries=[vector_query],
    top=3,
    search_fields=["name", "description", "imageCaption", "imageDescription", "imageTags"],
    select=["id", "name", "brand", "price", "description", "imageUrl", "imageCaption", "imageTags"]
)

print(f"🔍 검색어: '{search_query}'")
print(f"🔗 검색 방식: 하이브리드 (키워드 + 벡터)")

display_search_results(results, "하이브리드 검색 결과")

🔍 검색어: '검정색 가죽 가방 찾아줘'
🔗 검색 방식: 하이브리드 (키워드 + 벡터)

[하이브리드 검색 결과]

1. 숄더백 (어바에센셜)
   가격: 159,000원
   검색 점수: 0.03333333507180214
   💬 이미지 캡션: A black leather quilted handbag with gold accents.
   🏷️  태그: bag, leather, black, quilted, gold
   📝 설명: 심플한 디자인의 가죽 숄더백입니다. 수납공간이 넉넉하고 일상 사용에 편리합니다....



2. 가죽 크로스백 (럭셔리레더)
   가격: 189,000원
   검색 점수: 0.032786883413791656
   💬 이미지 캡션: 오렌지 컬러의 가죽과 라탄 소재로 만든 가방.
   🏷️  태그: 가방, 디자인, 라탄, 가죽, 오렌지
   📝 설명: 고급 천연 가죽으로 제작된 크로스백입니다. 심플한 디자인으로 어떤 스타일과도 잘 어울립니다....



3. 캔버스 백팩 (어반트래블)
   가격: 79,000원
   검색 점수: 0.0320020467042923
   💬 이미지 캡션: 검은색 백팩이 바닥에 놓여있다.
   🏷️  태그: 백팩, 가방, 검은색, 심플, 로고
   📝 설명: 내구성이 뛰어난 캔버스 소재의 백팩입니다. 노트북 수납 공간이 있어 실용적입니다....


## 6. 복합 검색 예시

검색 범위: name, description, brand, imageCaption, imageDescription, imageTags + 벡터 임베딩

In [12]:
##############################################
# 9. 복합 검색 - 제품 설명과 이미지 정보 모두 활용
##############################################
search_query = "스타일리시한 도시용 백팩"

# 벡터 검색 추가
query_vector = get_embedding(search_query)
vector_query = VectorizedQuery(
    vector=query_vector,
    k_nearest_neighbors=5,
    fields="descriptionVector"
)

results = search_client.search(
    search_text=search_query,
    vector_queries=[vector_query],
    top=3,
    search_fields=["name", "description", "brand", "imageCaption", "imageDescription", "imageTags"],
    select=["id", "name", "brand", "price", "description", "imageUrl", "imageCaption", "imageDescription", "imageTags"]
)

print(f"🔍 검색어: '{search_query}'")
print(f"📚 검색 필드: 모든 텍스트 필드 (제품 정보 + 이미지 분석 결과)")
print(f"🔗 검색 방식: 하이브리드 (키워드 + 벡터)")

print(f"\n{'='*80}")
print(f"[복합 검색 결과]")
print(f"{'='*80}\n")

for idx, result in enumerate(results, 1):
    score = result.get('@search.score', 'N/A')
    
    print(f"{idx}. {result['name']} ({result['brand']}) - {result['price']:,.0f}원")
    print(f"   검색 점수: {score}")
    
    print(f"\n   📝 제품 설명:")
    print(f"      {result['description']}")
    
    if result.get('imageCaption'):
        print(f"\n   💬 이미지 캡션:")
        print(f"      {result['imageCaption']}")
    
    if result.get('imageDescription'):
        print(f"\n   🖼️  이미지 상세 설명:")
        print(f"      {result['imageDescription'][:150]}...")
    
    if result.get('imageTags'):
        print(f"\n   🏷️  이미지 태그:")
        print(f"      {', '.join(result['imageTags'])}")
    
    if result.get('imageUrl'):
        display(HTML(f'<img src="{result["imageUrl"]}" width="300" style="margin: 10px 0; border-radius: 8px;" />'))
    
    print("\n" + "-"*80 + "\n")

🔍 검색어: '스타일리시한 도시용 백팩'
📚 검색 필드: 모든 텍스트 필드 (제품 정보 + 이미지 분석 결과)
🔗 검색 방식: 하이브리드 (키워드 + 벡터)

[복합 검색 결과]

1. 캔버스 백팩 (어반트래블) - 79,000원
   검색 점수: 0.03333333507180214

   📝 제품 설명:
      내구성이 뛰어난 캔버스 소재의 백팩입니다. 노트북 수납 공간이 있어 실용적입니다.

   💬 이미지 캡션:
      검은색 백팩이 바닥에 놓여있다.

   🖼️  이미지 상세 설명:
      이미지에는 깔끔한 디자인의 검은색 백팩이 밝은 배경의 방에 놓여 있는 모습이 담겨 있다. 백팩은 심플한 외형 디자인과 앞면에 작고 노란색 로고가 특징이다....

   🏷️  이미지 태그:
      백팩, 가방, 검은색, 심플, 로고, 바닥, 밝은배경



--------------------------------------------------------------------------------

2. 숄더백 (어바에센셜) - 159,000원
   검색 점수: 0.032786883413791656

   📝 제품 설명:
      심플한 디자인의 가죽 숄더백입니다. 수납공간이 넉넉하고 일상 사용에 편리합니다.

   💬 이미지 캡션:
      A black leather quilted handbag with gold accents.

   🖼️  이미지 상세 설명:
      이 가방은 고급스러운 블랙 컬러의 가죽 재질로 만들어졌으며, 매력적인 퀼팅 디자인과 금색 체인이 특징입니다. 전면 중앙에는 두 개의 금색 G 문양 장식이 있고, 크로스바디 스트랩은 금속 체인과 가죽이 혼합되어 있습니다....

   🏷️  이미지 태그:
      bag, leather, black, quilted, gold, fashion, accessory



--------------------------------------------------------------------------------

3. 가죽 크로스백 (럭셔리레더) - 189,000원
   검색 점수: 0.016129031777381897

   📝 제품 설명:
      고급 천연 가죽으로 제작된 크로스백입니다. 심플한 디자인으로 어떤 스타일과도 잘 어울립니다.

   💬 이미지 캡션:
      오렌지 컬러의 가죽과 라탄 소재로 만든 가방.

   🖼️  이미지 상세 설명:
      이 가방은 상단이 가죽 소재로 제작되고 하단이 라탄 직조로 이루어진 하이브리드 디자인을 특징으로 합니다. 오렌지 색상으로, 세련되고 자연적인 매력을 동시에 지닙니다. 손잡이와 스트랩이 있으며, 전면에는 금속 장식이 포인트로 자리잡고 있습니다....

   🏷️  이미지 태그:
      가방, 디자인, 라탄, 가죽, 오렌지, 패션, 금속 장식



--------------------------------------------------------------------------------

